# Smart Attendance System using OpenCv and Facial Recognition

> Key features 
```txt
1. Recognizes images with help of Face recognition and image encoding. 
2. Gives us options to mark attendance for various subjects. 
3. Uses Tkinter library to render the GUI.
4. The attendance gets stored in a CSV file with the name of the student along with the currrent timestamp.
5. The name of the CSV file contains the subject name, combined with the current date.
6. There is no overlapping of the name of the students in the same CSV file .
7. The attendance is marked as soon as the face gets recognized and it shows an unknown label if there is no matching of the faces 
8. The ImagesAttendance directory stores all the images that we are going to train for the attendance model.
```

In [9]:
from tkinter import *
from tkinter import ttk
from functools import partial
from tkinter import messagebox
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from datetime import date

# Attendance marking Code

cap = cv2.VideoCapture(0)

x = str(date.today())

# Image Encoding

# image encoding helps us to extract the rgb colors of a image
# using the encoding we can check if a face matches during the video capture


def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

# Marking attendance in csv file


def markAttendance(name):
    with open("E:\python\smart-attendane-system-using-python-main\pythonProject\Attendance/"+combo.get()+"_"+x+".csv", 'r+') as f:
        # helps in reading the names
        myDataList = f.readlines()
        # creates a namelist which incldues the names that are present in the list
        nameList = []
        for line in myDataList:
            #  splits the names by comma delimeter
            entry = line.split(',')
            # stores the name present in the list in the namelist array
            nameList.append(entry[0])

            # check if the name is already present in the namelist
        if name not in nameList:
            #  find the current datetime using datetime library
            now = datetime.now()
            # specifiy the foramt in which you want to write the datetime
            dtString = now.strftime('%H:%M:%S')
            #  if the name is not found in the namelist then it is added in the csv(comma separated values) file using the below format
            # writelines helps us to write data in the csv file
            f.writelines(f'\n{name},{dtString}')


# Face Capture


def startprogram():

    # to capture the video
    cap = cv2.VideoCapture(0)

    #  this contains the path of our images
    path = 'E:\python\smart-attendane-system-using-python-main\pythonProject\ImagesAttendance'
    #  creating an empty image array to store the details of the facial recognition
    images = []
    # // simply contains the names of the students
    classNames = []
    myList = os.listdir(path)
    print(myList)
    # helps in finding names from the image files
    for cl in myList:
        curImg = cv2.imread(f'{path}/{cl}')
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])
    print(classNames)

    encodeListKnown = findEncodings(images)
    print('Encoding Complete')

    f = open("E:\python\smart-attendane-system-using-python-main\pythonProject\Attendance/" +
             combo.get()+"_"+x+".csv", 'w')
    f.write("Name,Timestamp")
    f.close()
    while True:
        success, img = cap.read()
        # img = captureScreen()
        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(
                encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(
                encodeListKnown, encodeFace)
            # print(faceDis)
            matchIndex = np.argmin(faceDis)
            #  if the face marking distances is familiar to other images then we can say that there is a matching
            #  if there is a match then we can mark the attendance of that studnet
            if faceDis[matchIndex] < 0.50:
                name = classNames[matchIndex].upper()
                markAttendance(name)
            else:
                name = 'Unknown'
            # print(name)
            #  corrdinates of the face
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2),
                          (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('Webcam', img)


# To stop the attendance
        if cv2.waitKey(1) & 0xFF == ord('F'):
            cap.release()
            cv2.destroyAllWindows()
            break

# GUI CODE

# closing attendance window


def on_closing2():
    tkWindow2.withdraw()
    tkWindow.deiconify()

# closing main login window


def on_closing1():
    x = messagebox.askquestion('askquestion', 'Do you want to exit ?')
    if x == 'yes':
        tkWindow.destroy()
        tkWindow2.destroy()

# attendance page show


def New_Window():
    tkWindow.withdraw()
    tkWindow2.deiconify()

# login button method

# helps to check if there is valid login from the user


def validateLogin(username, password):
    if username.get() == "admin":
        if password.get() == "admin":
            print("LOGGED IN SUCCESSFULY")
            usernameEntry.delete(0, END)
            passwordEntry.delete(0, END)
            New_Window()
        else:
            messagebox.showwarning(
                "showwarning", "Wrong username or password !")
            usernameEntry.delete(0, END)
            passwordEntry.delete(0, END)
    else:
        messagebox.showwarning("showwarning", "Wrong username or password !")
        usernameEntry.delete(0, END)
        passwordEntry.delete(0, END)
    return

# attendance button method


def finalPage():
    print("Attendance to be marked")
    startprogram()
    messagebox.showinfo("showinfo", "Attendance Recorded !")


# window1
tkWindow = Tk()
tkWindow.geometry('300x300')
tkWindow.title('Attendance System')
tkWindow.protocol("WM_DELETE_WINDOW", on_closing1)

# window2
tkWindow2 = Tk()
tkWindow2.geometry('300x300')
tkWindow2.withdraw()
tkWindow2.title('Disruptive Technology Attendance System')
tkWindow2.protocol("WM_DELETE_WINDOW", on_closing2)

# username label and text entry box
usernameLabel = Label(tkWindow, text="User Name").place(x=17, y=20)
username = StringVar()
usernameEntry = Entry(tkWindow, textvariable=username)
usernameEntry.place(x=90, y=20)

# password label and password entry box
passwordLabel = Label(tkWindow, text="Password").place(x=20, y=50)
password = StringVar()
passwordEntry = Entry(tkWindow, textvariable=password, show='*')
passwordEntry.place(x=90, y=50)

validateLogin = partial(validateLogin, username, password)

# login button
loginButton = Button(tkWindow, text="Login",
                     command=validateLogin).place(x=100, y=100)

# attendance button
attendancebutton = Button(
    tkWindow2, text="Start Attendance", command=finalPage).place(x=77, y=105)

subjects = Label(tkWindow2, text="Select the subject :").place(x=87, y=45)

# create a drop down list
sizes = [
    "CS", "Maths", "DT", "Physics", "BEEE", "ITPS"
]

combo = ttk.Combobox(tkWindow2, values=sizes, state="readonly")
combo.place(x=45, y=71)
combo.current(0)


# stop attendance
quitLable = Label(
    tkWindow2, text="Press F to stop attendance").place(x=68, y=135)

tkWindow.mainloop()


ModuleNotFoundError: No module named 'face_recognition'